In [ ]:
import cv2
import numpy as np
from fpdf import FPDF

def scan_document(image_path, output_path, output_format='jpg'):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform edge detection using Canny
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours in the edge image
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort the contours by area in descending order
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Assume the largest contour is the document
    document_contour = contours[0]

    # Calculate the perimeter of the contour
    perimeter = cv2.arcLength(document_contour, True)

    # Approximate the polygonal curve of the contour
    approx = cv2.approxPolyDP(document_contour, 0.02 * perimeter, True)
    

    # Check if the contour approximation contains exactly 4 points
    if len(approx) != 4:
        print("Unable to detect a quadrilateral shape for the document.")
        return

    # Reorder the points of the polygonal curve
    pts = np.float32(approx.reshape(4, 2))
    rect = np.zeros((4, 2), dtype="float32")

    # Calculate the top-left, top-right, bottom-right, and bottom-left points of the rectangle
    s = np.sum(pts, axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # Calculate the width and height of the rectangle
    (tl, tr, br, bl) = rect
    width = max(np.linalg.norm(tr - tl), np.linalg.norm(br - bl))
    height = max(np.linalg.norm(bl - tl), np.linalg.norm(br - tr))

    # Create a destination rectangle with the desired width and height
    dst = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype="float32")

    # Calculate the perspective transform matrix and apply it to get a top-down view of the document
    M = cv2.getPerspectiveTransform(rect, dst)
    scanned = cv2.warpPerspective(image, M, (int(width), int(height)))

    # Save the scanned document as an image or PDF file
    if output_format == 'jpg':
        cv2.imwrite(output_path, scanned)
        print("Scanned document saved as JPEG:", output_path)
    elif output_format == 'pdf':
        pdf = FPDF()
        pdf.add_page()
        pdf.image(output_path, x=10, y=10, w=190)
        pdf.output(output_path)
        print("Scanned document saved as PDF:", output_path)
    else:
        print("Invalid output format.")

# Specify the input image path and output file path
image_path = "doc2.jpg"
output_path = "scanned_document.jpg"
output_format = 'jpg'  

# Call the function to scan the document and save it
scan_document(image_path, output_path, output_format)
